In [1]:
import re
import os
import time
import random
import psutil
import sqlite3
import datetime
import pandas as pd
from time import sleep
from emoji import emojize
from selenium import webdriver
from IPython.display import clear_output
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

# FUNÇÕES DE MANIPULAÇÃO DAS PERGUNTAS E SELENIUM

In [2]:
#função que recebe uma lista de perguntas em um txt e escolhe uma de forma aleatoria, retorna a pergunta e seu indice
def pergunta(nivel, repeticao):
    # Lê o arquivo com o pandas
    df = pd.read_csv(nivel, sep='§', header=None, encoding='utf-8')

    # Remove espaços em branco no início e no final das linhas
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Converte o dataframe em uma lista
    lines = df.values.tolist()

    # Remove as listas vazias geradas pelo separador '§'
    lines = [line for line in lines if len(line) > 0]

    # Sorteia uma linha aleatória que não esteja no conjunto de repetição
    while True:
        random_index = random.randint(0, len(lines) - 1)
        if random_index not in repeticao:
            break

    # Retorna a linha escolhida e o índice
    return lines[random_index][0], random_index

#função de iteração com os usuarios, onde ele envia mensagens de iteração de forma aleatoria, mas com probabilidade de 40%
def iteracao(text):
    empty = []
    alternativa_1 = "O que vocês preferem:\n" #Tem 60% de chance de enviar apenas "O que vocês preferem:\n"
    alternativa_2 = pergunta(text, empty)[0] #Tem 40% de chance de enviar uma mensagem iterativa
    
    #escolhe
    escolha = random.choices(
        [alternativa_1, alternativa_2],
        weights=[0.6, 0.4],
        k=1
    )
    
    return escolha[0]

#função que envia uma mensagem para os contatos/grupos do WhatsApp através do Selenium WebDriver. 
def BOT_MENSAGEM(out):
    mensagem = driver.find_element("xpath", '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div[1]/p')
    mensagem.click()
    mensagem.send_keys(out + Keys.ENTER)
    
#É necessario manter a privacidade, portanto, está função garante excluir todas as mensagens do contato/grupo
def EXCLUIR_MENSAGENS():
    button_menu = driver.find_element_by_xpath('//*[@id="main"]/header/div[3]/div/div[2]/div/div/span')
    button_menu.click()
    time.sleep(0.5) #espera pagina carregar
    button_delet = driver.find_element_by_xpath("//li[@data-testid='mi-clear']//div[@aria-label='Limpar mensagens']")
    button_delet.click()
    time.sleep(0.5) #espera pagina carregar
    confirm_deletion = driver.find_element_by_css_selector('button[data-testid="popup-controls-ok"]')
    confirm_deletion.click()
    time.sleep(0.1)
    
#simples função de simples animação de espera para fins de visualização do desenvolvedor
def WAITING():
    animation = [".", "..", "...", " ..", "  .", "  ..", "  ...", "  ..", "  .", " ..", "...", "..", "."]
    
    for i in animation:
        print(i)
        time.sleep(0.1)
        clear_output(wait=True)

# INICIALIZAÇÃO DO SELENIUM WebDriver DO SITE WHATSAPP WEB

In [10]:
driver = webdriver.Chrome('./chromedriver')
driver.get('https://web.whatsapp.com/')
# Aguarda o usuário escanear o QR code
input("Aperte Enter após escanear o QR code...")

Aperte Enter após escanear o QR code...


''

# FUNÇÕES DE MANIPULAÇÃO DO BANCO DE DADOS SQLite

In [4]:
def adicionar_contato(nome, nivel1='', nivel2='', nivel3='', nivel4='', publico=0, adulto=0):
    conn = sqlite3.connect('dados_contatos_grupos.db')
    cursor = conn.cursor()
    
    cursor.execute("INSERT INTO contatos (nome, nivel1, nivel2, nivel3, nivel4, publico, adulto) VALUES (?, ?, ?, ?, ?, ?, ?)", (nome, nivel1, nivel2, nivel3, nivel4, publico, adulto))

    conn.commit()
    conn.close()
    
    print(f"Contato {nome} adicionado com sucesso!")
    
def atualizar_perguntas(nome, nivel, ultimo_indice):
    conn = sqlite3.connect('dados_contatos_grupos.db')
    cursor = conn.cursor()
    
    cursor.execute(f"SELECT nivel{nivel} FROM contatos WHERE nome=?", (nome,))
    perguntas = cursor.fetchone()[0]
    perguntas = perguntas.split(',') if perguntas else []
    perguntas.append(str(ultimo_indice))
    perguntas_str = ','.join(perguntas)
    
    cursor.execute(f"UPDATE contatos SET nivel{nivel}=? WHERE nome=?", (perguntas_str, nome))
    
    conn.commit()
    conn.close()

def contato_existe(nome):
    conn = sqlite3.connect('dados_contatos_grupos.db')
    cursor = conn.cursor()
    cursor.execute("SELECT count(*) FROM contatos WHERE nome=?", (nome,))
    resultado = cursor.fetchone()
    conn.close()
    return resultado[0] > 0

def obter_perguntas(nome, nivel):
    conn = sqlite3.connect('dados_contatos_grupos.db')
    cursor = conn.cursor()

    cursor.execute(f"SELECT nivel{nivel} FROM contatos WHERE nome=?", (nome,))
    perguntas_str = cursor.fetchone()[0]

    if not perguntas_str:
        return []

    perguntas = [int(x) for x in perguntas_str.split(",")]
    
    return perguntas

def obter_adulto(nome):
    conn = sqlite3.connect('dados_contatos_grupos.db')
    cursor = conn.cursor()

    cursor.execute("SELECT adulto FROM contatos WHERE nome=?", (nome,))
    adulto = cursor.fetchone()[0]
    
    return adulto

def atualizar_adulto(nome, novo_valor):
    conn = sqlite3.connect('dados_contatos_grupos.db')
    cursor = conn.cursor()

    cursor.execute("UPDATE contatos SET adulto=? WHERE nome=?", (novo_valor, nome))
    
    conn.commit()
    
def atualizar_ultima_mensagem(nome, mensagem):
    conn = sqlite3.connect('dados_contatos_grupos.db')
    cursor = conn.cursor()

    agora = datetime.datetime.now().strftime('%S')
    ultima_mensagem = f"{mensagem}:{agora}"

    cursor.execute(f"UPDATE contatos SET ultima_mensagem = ? WHERE nome = ?", (ultima_mensagem, nome))
    conn.commit()

    conn.close()

def obter_ultima_mensagem(nome):
    conn = sqlite3.connect('dados_contatos_grupos.db')
    cursor = conn.cursor()

    cursor.execute("SELECT ultima_mensagem FROM contatos WHERE nome=?", (nome,))
    ultima_mensagem = cursor.fetchone()[0]
    
    return ultima_mensagem

def incrementa_numero_mensagens(nome):
    conn = sqlite3.connect('dados_contatos_grupos.db')
    c = conn.cursor()
    c.execute('UPDATE contatos SET numero_de_mensagens = numero_de_mensagens + 1 WHERE nome = ?', (nome,))
    conn.commit()
    conn.close()

# zera o valor da coluna "numero_de_mensagens" para um determinado título
def zera_numero_mensagens(nome):
    conn = sqlite3.connect('dados_contatos_grupos.db')
    c = conn.cursor()
    c.execute('UPDATE contatos SET numero_de_mensagens = 0 WHERE nome = ?', (nome,))
    conn.commit()
    conn.close()
    
def get_numero_de_mensagens(nome_contato):
    # Conecta ao banco de dados
    conn = sqlite3.connect('dados_contatos_grupos.db')

    # Cria um cursor
    cursor = conn.cursor()

    # Executa a consulta SQL para obter o valor da coluna "numero_de_mensagens" do contato com o nome especificado
    cursor.execute('SELECT numero_de_mensagens FROM contatos WHERE nome = ?', (nome_contato,))

    # Obtém o resultado da consulta
    result = cursor.fetchone()

    # Fecha a conexão com o banco de dados
    conn.close()
    
    return result[0]

In [12]:
def manipula(nivel, title, dinamic):
    MAX_MSG = 200
    question_n = pergunta(nivel, obter_perguntas(title, 1))
    atualizar_perguntas(title, 1, question_n[1])
    
    if dinamic is not None:
        BOT_MENSAGEM(iteracao(dinamic) + "\n" + question_n[0])
    else:
        BOT_MENSAGEM("O QUE VOCÊ PREFERE?" + "\n" + question_n[0])
        
    incrementa_numero_mensagens(title)
    if get_numero_de_mensagens(title) >= MAX_MSG:
        EXCLUIR_MENSAGENS()
        zera_numero_mensagens(title)
    #Entra no modo espera
    
    standby = driver.find_element_by_css_selector(
        ".lhggkp7q.ln8gz9je.rx9719la[style*='transform: translateY(0px)']")
    standby.click()
    time.sleep(0.04)

# FUNÇÃO DO GAME

In [6]:
def Game_OQVCP():
    regras = "As regras funcionam da seguinte forma:" + Keys.SHIFT + Keys.ENTER + Keys.SHIFT + "Você pode pedir uma pergunta normal, ousada, quente ou extremo" + Keys.SHIFT + Keys.ENTER + Keys.SHIFT + "Escreva \'normal\' :aur" + Keys.ENTER + Keys.SHIFT + Keys.ENTER + Keys.SHIFT + "Escreva \'ousado\' :new " + Keys.ENTER + Keys.SHIFT + Keys.ENTER + Keys.SHIFT + "Escreva \'quente\' :18" + \
        Keys.ENTER + Keys.SHIFT + Keys.ENTER + Keys.SHIFT + "Escreva \'extremo\' :oss" + Keys.ENTER + Keys.SHIFT + Keys.ENTER + Keys.SHIFT + "Então mandarei de forma aleatoria de acordo com seu pedido"
    rules = """As regras funcionam da seguinte forma:
        Você pode pedir uma pergunta normal, ousada, quente ou extremo
        Escreva 'normal' :aur
        Escreva 'ousado' :new 
        Escreva 'quente' :18
        Escreva 'extremo' :oss
        Então mandarei de forma aleatória de acordo com seu pedido"""
    #Entra no modo 'esperando nova mensagem'
    standby = driver.find_element_by_css_selector(
        ".lhggkp7q.ln8gz9je.rx9719la[style*='transform: translateY(0px)']")
    standby.click()
    
    title = ''
    publico = 0
    publicoE = 0
    messages = ''
    ult = messages
    
    print("GAME INICIADO")

    while True:
        time.sleep(1)

        #Quando uma mensagem da classe SPAN temporaria não existe, ela não pode clicar e gera erro, logo o erro é tratado.
        try:
            #Clica em uma nova mensagem, caso uma apareça
            new_msg = driver.find_element_by_css_selector(
                ".l7jjieqr.cfzgl7ar.ei5e7seu.h0viaqh7.tpmajp1w.c0uhu3dl.riy2oczp.dsh4tgtl.sy6s5v3r.gz7w46tb.lyutrhe2.qfejxiq4.fewfhwl7.ovhn1urg.ap18qm3b.ikwl5qvt.j90th5db.aumms1qt")
            new_msg.click()

            current_conversation = driver.find_element_by_xpath(
                '//div[@tabindex="0" and @aria-selected="true"]')

            # Encontre o elemento do título da conversa
            conversation_title = current_conversation.find_element_by_xpath(
                './/span[@class="ggj6brxn gfz4du6o r7fjleex g0rxnol2 lhj4utae le5p0ye3 l7jjieqr _11JPr"]')

            # Extraia o texto do título da conversa
            title = conversation_title.text

            #Verifica se o contato/grupo já existe no banco de dados
            if contato_existe(title):

                #Obtem ultima mensagem enviada no contato/grupo
                messages = driver.find_elements(
                    By.CSS_SELECTOR, '.message-in:last-child')
                ult = messages[len(messages) - 1].text  # Ultima mensagem
            else:

                #Salva os contatos/grupos novos no banco de dados
                adicionar_contato(title, '', '', '', '', 0, 0)
                print("O contato/grupo " + title +
                      " foi adicionado ao banco de dados")
                BOT_MENSAGEM("Bem Vindo ao BOTLE, segue as regras:")
                BOT_MENSAGEM(regras)
                continue

        except NoSuchElementException:
            #print("MODO ESPERANDO NOVA MENSAGEM")
            #Entra em modo espera caso nenhuma nova mensagem apareça
            #A ultima mensagem é atualizada para um codigo aleatorio, apenas para fins praticos
            ult = '5dF*7w#z8Lk@2'
            standby = driver.find_element_by_css_selector(
                    ".lhggkp7q.ln8gz9je.rx9719la[style*='transform: translateY(0px)']")
            standby.click()
        
        if "Você" not in ult and any(word in ult.lower() for word in ["normal", "ousad", "quent", "extrem", "regras", "estou ciente"]):

            if "extrem" in ult.lower() and obter_adulto(title) == 0:
                BOT_MENSAGEM("As perguntas extremas podem ser desconfortáveis ou ofensivas para algumas pessoas. *Se você for maior de 18 anos e estiver disposto a enfrentar possíveis riscos, digite ESTOU CIENTE para continuar.* Lembre-se de que este é apenas um jogo e as perguntas não devem ser levadas para a vida real. Caso se sinta desconfortável em qualquer momento, sinta-se à vontade para sair do jogo ou pular as perguntas.")
                atualizar_ultima_mensagem(title, "extrem")

                #Entra no modo espera
                standby = driver.find_element_by_css_selector(
                    ".lhggkp7q.ln8gz9je.rx9719la[style*='transform: translateY(0px)']")
                standby.click()
                continue
            elif "extrem" in obter_ultima_mensagem(title) and "estou ciente" in ult.lower():
                atualizar_adulto(title, 1)
                BOT_MENSAGEM("OK! Sua resposta foi salva, a partir de agora todas mensagens *EXTREMAS* estão disponiveis neste contato/grupo. Por favor, peça uma pergunta EXTREMA. Lembre-se de que este é apenas um jogo e as perguntas não devem ser levadas para a vida real. Caso se sinta desconfortável em qualquer momento, sinta-se à vontade para sair do jogo ou pular as perguntas.")

                #Entra no modo espera
                standby = driver.find_element_by_css_selector(
                    ".lhggkp7q.ln8gz9je.rx9719la[style*='transform: translateY(0px)']")
                standby.click()

            if "normal" in ult.lower():
                manipula("NIVEL_1.txt", title, "normal.txt")
            
            elif "ousad" in ult.lower():
                manipula("NIVEL_2.txt", title, "ousado.txt")
                
            elif "quent" in ult.lower():
                if publico == 1:
                    BOT_MENSAGEM("Identifiquei que este grupo é público. De acordo com os termos e política de nosso Bot, as perguntas da categoria *quente* estão somente disponíveis em grupos privados. Mas você pode continuar a pedir as *normais* e *ousadas*. Agradeço a compreensão de todos.")

                    #Entra no modo espera
                    standby = driver.find_element_by_css_selector(
                        ".lhggkp7q.ln8gz9je.rx9719la[style*='transform: translateY(0px)']")
                    standby.click()
                else:
                    manipula("NIVEL_3.txt", title, "quente.txt")
            
            elif "extrem" in ult.lower():
                if publicoE == 1:
                    #BOT_MENSAGEM("Identifiquei que este grupo é público. De acordo com os termos e política de nosso Bot, as perguntas da categoria *extremas* estão somente disponíveis em grupos privados. Mas você pode continuar a pedir as *normais* e *ousadas*. Agradeço a compreensão de todos.")
                    BOT_MENSAGEM(extremas_resposta_inativo)

                    #Entra no modo espera
                    standby = driver.find_element_by_css_selector(
                        ".lhggkp7q.ln8gz9je.rx9719la[style*='transform: translateY(0px)']")
                    standby.click()

                else:
                    manipula("NIVEL_4.txt", title, None)

            elif "regras" in ult.lower():
                BOT_MENSAGEM(rules)
                
                standby = driver.find_element_by_css_selector(
                    ".lhggkp7q.ln8gz9je.rx9719la[style*='transform: translateY(0px)']")
                standby.click()

        elif "desligar_pdnejoh_bot" in ult:

            BOT_MENSAGEM("DESLIGANDO TODO O SISTEMA")
            print("O sistema foi desligado")

            return

# Simples codigo com tratamento de erro
Durante muitos testes com vários usuários, observei que em algumas ocasiões ocorria um erro no Selenium devido à página não carregar rapidamente o suficiente, apesar de o tempo de espera ter sido estabelecido como 1 segundo, que já é o tempo mínimo. Acredito que a razão disso seja a memória do computador. Esses erros exigiam intervenção manual para executar a função Game_OQVCP() novamente. No entanto, agora essa ação não é mais necessária. Além disso, o erro é tratado e o código espera 5 segundos para que o computador carregue novamente as páginas. Por fim, o erro é salvo em um arquivo erros.txt com a data e hora da ocorrência para fins de analise.

In [13]:
bugs = 0 #Caso ocorra um grande limites de bugs, o bot envia uma mensagem para que o usuario contacte o desenvolvedor
while True:
    try:
        Game_OQVCP()
        
    except Exception as e:
        bugs += 1
        agora = datetime.datetime.now()
        data_hora_formatada = agora.strftime("%d/%m/%Y %H:%M:%S")
        mensagem_erro = f"Ocorreu uma exceção em {data_hora_formatada}: {e}\n"
        with open("erros.txt", "a") as arquivo:
            arquivo.write(mensagem_erro)
            
        if bugs >= 10:
            BOT_MENSAGEM("Esses erros estão frequentes, contacta meu desenvolvedor e informa o bug a ele, tá aqui o whats dele number_phone")
            BOT_MENSAGEM("Mesmo assim vou tentar reiniciar novamente, aguarda 5 segundos por favor")
            time.sleep(5)
            BOT_MENSAGEM("Agora tomei logo 3 Ciberozil, cuidado se não entro em coma cibernetico hehe, pede ai")
        else:
            print(f"Ocorreu uma exceção: {e}")
            BOT_MENSAGEM("Ohh, parece que não estou bem, meu sistema deu bug e doeu, sorry, espera 5 segundos, enquanto eu reinicio meu algoritmo")
            #time.sleep(5)
            #BOT_MENSAGEM("50% reiniciado, tá quase")
            time.sleep(1)
            for i in range(1, 6):
                time.sleep(1)
                porcentagem = i * 20
                mensagem = f"{porcentagem}% concluído"
                BOT_MENSAGEM(mensagem)
            BOT_MENSAGEM("Pronto, tomei um Ciberozil e me sinto pleno, pede de novo!")
            # Se ocorreu um erro, reinicia a função
        continue

GAME INICIADO


KeyboardInterrupt: 

In [ ]:
comando = ''

while True:
    #search_box = driever.find_element_by_xpath('//div[@class="_2_1wd copyable-text selectable-text"]')
    #search_box.send_keys("Jack")

    
    contact = driver.find_element_by_xpath('//span[@title="Jack"]')
    contact.click()
    messages = driver.find_elements(By.CSS_SELECTOR, '.message-in:last-child')
    comando = messages[len(messages) - 1].text if len(messages) > 0 else None
    #print(comando)
    WAITING()
    if "ligar_bot" in comando:
        comando = ''
        BOT_MENSAGEM("JOGO LIGADO!")
        Game_OQVCP()
    elif "exit_the_system_botle" in comando:
        break
        
if "exit_the_system_83999314480" in comando:
    os.system("shutdown /s /t 0")

'\n//*[@id="main"]/header/div[3]/div/div[2]/div/div/span\n//*[@id="main"]/header/div[3]/div/div[2]/div/div/span\n//*[@id="app"]/div/span[4]/div/ul/div/div/li[4]/div\n//*[@id="app"]/div/span[4]/div/ul/div/div/li[4]/div\n//*[@id="main"]/header/div[3]/div/div[2]/div/div/span\n'